In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:1"

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 32, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0.2):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(2, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
        
        self.lineartest0 = nn.Linear(2, emsize)
        self.lineartest1 = nn.Linear(2, emsize)
        self.lineartest2 = nn.Linear(2, emsize)
        self.lineartest3 = nn.Linear(2 * emsize, emsize)
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 2]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def test_encode(self, src): #src must be [batchsize * nsamples, npoints = 5, 2]
        point_1 = self.lineartest1(src[:, 3, :] - 0.5) #want mean 0!!!!
        point_2 = self.lineartest1(src[:, 4, :] - 0.5)
        remaining = self.lineartest0(src[:, :3, :]) #[batchsize * nsamples, 3, emsize]
        point_1_message = self.lineartest1(src[:, 3, :] - 0.5).squeeze(1)
        point_2_message = self.lineartest2(src[:, 4, :] - 0.5).squeeze(1)
        point_1 = F.relu(torch.cat([point_1, point_2_message], dim = 1))
        point_2 = F.relu(torch.cat([point_2, point_1_message], dim = 1))
        point_1 = self.lineartest3(point_1)
        point_2 = self.lineartest3(point_2)
        src = torch.cat([torch.zeros_like(remaining.transpose(0, 1)), point_1.unsqueeze(0), point_2.unsqueeze(0)])
        return src #[npoints = 5, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 3], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 3, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 3, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 3, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 3, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 3, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, dtype = torch.bool, device = device).triu(1) #[npoints - 3, npoints - 3], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 3, npoints - 3]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 3, npoints - 3]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment():
    def reset(self, npoints, batchsize, nsamples):
        if npoints <= 3:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = batchsize * nsamples #so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = torch.rand([batchsize, npoints - 3, 2], device = device).unsqueeze(1).expand(-1, nsamples, -1, -1).reshape(self.batchsize, npoints - 3, 2)
        self.corner_points = torch.tensor([[0, 0], [2, 0], [0, 2]], dtype = torch.float, device = device)
        self.points = torch.cat([self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1), self.points], dim = -2) #[batchsize * nsamples, npoints, 2]
        self.points_mask = torch.cat([torch.ones([self.batchsize, 3], dtype = torch.bool, device = device), torch.zeros([self.batchsize, npoints - 3], dtype = torch.bool, device = device)], dim = 1)
        self.points_sequence = torch.empty([self.batchsize, 0], dtype = torch.long, device = device)
        
        """use a trick, for the purpose of an 'external' triangle that is always left untouched, which means we don't have to deal with boundary edges as being different. external triangle is [0, 1, 2] traversed clockwise..."""
        self.partial_delaunay_triangles = torch.tensor([[0, 2, 1], [0, 1, 2]], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1, -1).contiguous() #[batchsize, ntriangles, 3] contains index of points, always anticlockwise
        self.partial_delaunay_edges = torch.tensor([5, 4, 3, 2, 1, 0], dtype = torch.int64, device = device).unsqueeze(0).expand(self.batchsize, -1).contiguous() #[batchsize, ntriangles * 3] contains location of corresponding edge (edges go in order 01, 12, 20). Edges will always flip since triangles are stored anticlockwise.
        
        self.ntriangles = 2 #can store as scalar, since will always be the same
        self.cost = torch.zeros([self.batchsize], device = device)
        
        self.logprob = torch.zeros([self.batchsize], device = device, requires_grad = True)
    
    def update(self, point_index): #point_index is [batchsize]
        if point_index.size(0) != self.batchsize:
            print("Error: point_index.size() doesn't match expected size, should be [batchsize]")
            return
        if self.points_mask.gather(1, point_index.unsqueeze(1)).sum():
            print("Error: some points already added")
            return
        triangles_coordinates = self.points.gather(1, self.partial_delaunay_triangles.view(self.batchsize, self.ntriangles * 3).unsqueeze(2).expand(-1, -1, 2)).view(self.batchsize, self.ntriangles, 3, 2) #[batchsize, ntriangles, 3, 2]
        newpoint = self.points.gather(1, point_index.unsqueeze(1).unsqueeze(2).expand(self.batchsize, 1, 2)).squeeze(1) #[batchsize, 2]
        
        incircle_matrix = torch.cat([triangles_coordinates, newpoint.unsqueeze(1).unsqueeze(2).expand(-1, self.ntriangles, 1, -1)], dim = -2) #[batchsize, ntriangles, 4, 2]
        incircle_matrix = torch.cat([incircle_matrix, (incircle_matrix * incircle_matrix).sum(-1, keepdim = True), torch.ones([self.batchsize, self.ntriangles, 4, 1], device = device)], dim = -1) #[batchsize, ntriangles, 4, 4]
        incircle_test = incircle_matrix.det() > 0 #[batchsize, ntriangles], is True if inside incircle
        removed_edge_mask = incircle_test.unsqueeze(2).expand(-1, -1, 3).reshape(-1) #[batchsize * ntriangles * 3]
        
        edges = (self.partial_delaunay_edges + self.ntriangles * 3 * torch.arange(self.batchsize, device = device).unsqueeze(1)).view(-1) #[batchsize * ntriangles * 3]
        neighbouring_edge = edges.masked_select(removed_edge_mask)
        neighbouring_edge_mask = torch.zeros([self.batchsize * self.ntriangles * 3], device = device, dtype = torch.bool)
        neighbouring_edge_mask[neighbouring_edge] = True
        neighbouring_edge_mask = (neighbouring_edge_mask * removed_edge_mask.logical_not()) #[batchsize * ntriangles * 3]
        
        n_new_triangles = neighbouring_edge_mask.view(self.batchsize, -1).sum(-1) #[batchsize]
        
        new_point = point_index.unsqueeze(1).expand(-1, self.ntriangles * 3).masked_select(neighbouring_edge_mask.view(self.batchsize, -1))
        
        second_point_mask = neighbouring_edge_mask.view(self.batchsize, -1, 3) #[batchsize, ntriangles 3]
        (first_point_indices0, first_point_indices1, first_point_indices2) = second_point_mask.nonzero(as_tuple = True)
        first_point_indices2 = (first_point_indices2 != 2) * (first_point_indices2 + 1)
        
        first_point = self.partial_delaunay_triangles[first_point_indices0, first_point_indices1, first_point_indices2] #[?]
        second_point = self.partial_delaunay_triangles.masked_select(second_point_mask) #[?]
        
        new_triangles_mask = torch.cat([incircle_test, torch.ones([self.batchsize, 2], dtype = torch.bool, device = device)], dim = 1) #[batchsize, ntriangles + 2]
        
        new_neighbouring_edges = 3 * new_triangles_mask.nonzero(as_tuple = True)[1] #[?], 3* since is the 01 edge of new triangles (see later)
        self.partial_delaunay_edges.masked_scatter_(neighbouring_edge_mask.view(self.batchsize, -1), new_neighbouring_edges) #still [batchsize, ntriangles * 3] for now
        
        self.partial_delaunay_triangles = torch.cat([self.partial_delaunay_triangles, torch.empty([self.batchsize, 2, 3], dtype = torch.long, device = device)], dim = 1)
        self.partial_delaunay_edges = torch.cat([self.partial_delaunay_edges, torch.empty([self.batchsize, 6], dtype = torch.long, device = device)], dim = 1)
        new_triangles = torch.stack([first_point, second_point, new_point], dim = 1) #[?, 3], edge here is flipped compared to edge in neighbouring triangle (so first_point is the second point in neighbouring edge)
        self.partial_delaunay_triangles.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3), new_triangles) #[batchsize, ntriangles + 2, 3]
        
        new_edge01 = neighbouring_edge_mask.view(self.batchsize, -1).nonzero(as_tuple = True)[1] #[?]
        
        """we are currently storing which triangles have to be inserted, via the edges along the perimeter of the delaunay cavity, we need to compute which edge is to the 'left'/'right' of each edge"""
        """don't have the memory to do a batchsize * n * n boolean search, don't have the speed to do a batchsize^2 search (as would be the case for sparse matrix or similar)"""
        """best alternative: rotate the edge around right point, repeat until hit edge in mask (will never go to an edge of a removed triangle before we hit edge in mask) should basically be order 1!!!!!"""
        
        neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?]
        next_neighbouring_edge_index = torch.empty_like(neighbouring_edge_index) #[?]
        
        rotating_flipped_neighbouring_edge_index = neighbouring_edge_mask.nonzero(as_tuple = True)[0] #[?], initialise
        todo_mask = torch.ones_like(next_neighbouring_edge_index, dtype = torch.bool) #[?]
        while todo_mask.sum():
            rotating_neighbouring_edge_index = rotating_flipped_neighbouring_edge_index + 1 - 3 * (rotating_flipped_neighbouring_edge_index % 3 == 2) #[todo_mask.sum()], gets smaller until nothing left EFFICIENCY (this may be seriously stupid, as it requires making a bunch of copies when I could be doing stuff inplace)
            
            update_mask = neighbouring_edge_mask[rotating_neighbouring_edge_index] #[todo_mask.sum()]
            update_mask_unravel = torch.zeros_like(todo_mask).masked_scatter(todo_mask, update_mask) #[?]
            
            next_neighbouring_edge_index.masked_scatter_(update_mask_unravel, rotating_neighbouring_edge_index.masked_select(update_mask)) #[?]
            
            todo_mask.masked_fill_(update_mask_unravel, False) #[?]
            rotating_flipped_neighbouring_edge_index = edges[rotating_neighbouring_edge_index.masked_select(update_mask.logical_not())] #[todo_mask.sum()]
        triangle_index = new_triangles_mask.view(-1).nonzero(as_tuple = True)[0] #[?], index goes up to batchsize * (ntriangles + 2), this is needed for when we invert the permutation by scattering (won't scatter same number of triangles per batch)
        
        next_triangle_index = torch.empty_like(edges).masked_scatter_(neighbouring_edge_mask, triangle_index)[next_neighbouring_edge_index] #[?], index goes up to batchsize * (ntriangles + 2)
        next_edge = 3 * next_triangle_index + 1 #[?]
        
        invert_permutation = torch.empty_like(new_triangles_mask.view(-1), dtype=torch.long) #[batchsize * (ntriangles + 2)]
        invert_permutation[next_triangle_index] = triangle_index #[batchsize * (ntriangles + 2)]
        previous_triangle_index = invert_permutation.masked_select(new_triangles_mask.view(-1)) #[?]
        previous_edge = 3 * previous_triangle_index + 2 #[?]
        
        """in the above we rotated around 'first_point' in our new triangles"""
        new_edge20 = next_edge % ((self.ntriangles + 2) * 3) #[?]
        new_edge12 = previous_edge % ((self.ntriangles + 2) * 3) #[?]
        
        new_edges = torch.stack([new_edge01, new_edge12, new_edge20], dim = 1) #[?, 3]
        self.partial_delaunay_edges.masked_scatter_(new_triangles_mask.unsqueeze(2).expand(-1, -1, 3).reshape(self.batchsize, -1), new_edges) #[batchsize, (ntriangles + 2) * 3]
        
        self.ntriangles += 2
        """currently only count the extra triangles you replace (not the on you have to remove because you're located there, and not the ones you make because you have to create two more"""
        self.cost += (n_new_triangles - 3)
        self.points_mask.scatter_(1, point_index.unsqueeze(1).expand(-1, self.npoints), True)
        self.points_sequence = torch.cat([self.points_sequence, point_index.unsqueeze(1)], dim = 1)
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 300000, npoints = 13, batchsize = 2000, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(3, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 0.5
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                - logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                * positive_reinforcement_binary 
                + logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 11) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
  

In [ ]:
train(npoints = 23, batchsize = 1200, nsamples = 8)

35.74333572387695 -40.4248046875 -42.262054443359375 -40.66893005371094 -42.67074966430664 -42.67074966430664
35.18916702270508 -39.31804275512695 -42.0859375 -39.671871185302734 -43.21395492553711 -43.213958740234375
34.99333572387695 -38.44990158081055 -41.86281967163086 -38.1721076965332 -41.048423767089844 -41.04842758178711
34.86916732788086 -38.58427429199219 -41.913917541503906 -38.12294387817383 -41.735939025878906 -41.73593521118164
34.609169006347656 -38.07272720336914 -41.772193908691406 -38.21965789794922 -41.701560974121094 -41.701560974121094
34.23250198364258 -36.66769790649414 -41.32535934448242 -37.623939514160156 -40.447845458984375 -40.447845458984375
34.03333282470703 -34.61423873901367 -40.398414611816406 -35.237796783447266 -38.73758316040039 -38.737586975097656
33.54083251953125 -32.43416976928711 -39.187721252441406 -33.61690139770508 -37.071434020996094 -37.07143020629883
33.36833572387695 -32.51325988769531 -39.24564743041992 -31.861543655395508 -41.8573074340

32.44166564941406 -32.45647430419922 -39.73451614379883 -33.252052307128906 -40.108299255371094 -40.108299255371094
32.50666809082031 -33.07151794433594 -39.99108123779297 -34.21049499511719 -41.50611114501953 -41.50611114501953
32.47083282470703 -33.408287048339844 -40.11994171142578 -33.46706771850586 -42.27180480957031 -42.27180099487305
32.244998931884766 -33.509212493896484 -40.08403396606445 -36.13527297973633 -35.7867431640625 -35.786746978759766
32.22333526611328 -33.473175048828125 -39.95732116699219 -35.58271026611328 -37.58378601074219 -37.58378601074219
32.366668701171875 -33.205631256103516 -39.71870422363281 -32.49414825439453 -40.68533706665039 -40.68533706665039
32.40833282470703 -33.18425750732422 -39.61157989501953 -35.05782699584961 -38.67479705810547 -38.67479705810547
32.41583251953125 -32.85879898071289 -39.434593200683594 -32.958290100097656 -38.59821319580078 -38.59821319580078
32.366668701171875 -32.6638298034668 -39.30912399291992 -32.2930908203125 -39.3215980

32.1341667175293 -33.24582290649414 -39.46050262451172 -33.265933990478516 -37.38422393798828 -37.38422393798828
31.838335037231445 -33.190040588378906 -39.46217346191406 -33.94445037841797 -36.34806823730469 -36.34806823730469
31.895000457763672 -32.989463806152344 -39.37763214111328 -32.119266510009766 -40.100006103515625 -40.100006103515625
32.04083251953125 -32.73915481567383 -39.220306396484375 -32.45995330810547 -38.16693115234375 -38.166927337646484
31.79916763305664 -32.35686111450195 -39.07326889038086 -32.756134033203125 -36.003665924072266 -36.003665924072266
31.990001678466797 -32.48737716674805 -39.123748779296875 -33.9860954284668 -40.377193450927734 -40.37718963623047
31.90250015258789 -32.4576530456543 -39.101192474365234 -31.848234176635742 -38.05107116699219 -38.05107498168945
31.67500114440918 -32.461463928222656 -39.11138916015625 -34.682838439941406 -36.02387237548828 -36.02387237548828
31.836668014526367 -32.33169174194336 -39.070045471191406 -33.002315521240234 -

31.96583366394043 -32.336666107177734 -39.69087600708008 -30.783977508544922 -40.69136047363281 -40.69136047363281
31.850833892822266 -32.285064697265625 -39.710594177246094 -31.805816650390625 -41.75596618652344 -41.7559700012207
32.09166717529297 -32.34951400756836 -39.74055099487305 -31.08605194091797 -40.82555389404297 -40.82555389404297
32.21833419799805 -32.50234603881836 -39.81612777709961 -32.924354553222656 -36.82741928100586 -36.82741928100586
32.05083465576172 -32.617713928222656 -39.835628509521484 -30.143718719482422 -40.6744384765625 -40.6744384765625
32.128334045410156 -32.72335433959961 -39.94048309326172 -32.742279052734375 -36.00304412841797 -36.00304412841797
32.11333465576172 -32.6161003112793 -39.8761100769043 -30.926834106445312 -43.06201171875 -43.06201934814453
32.317501068115234 -32.630821228027344 -39.906837463378906 -31.763731002807617 -38.73853302001953 -38.7385368347168
32.331668853759766 -32.65363693237305 -39.943504333496094 -32.2056999206543 -36.44345474

32.00833511352539 -32.869842529296875 -40.150821685791016 -34.124267578125 -38.50080490112305 -38.50080108642578
32.11750030517578 -32.715789794921875 -40.10139465332031 -32.35136413574219 -39.73101043701172 -39.73100662231445
32.163333892822266 -32.66851043701172 -40.05286407470703 -31.69457244873047 -38.32461929321289 -38.32461929321289
31.71500015258789 -32.57209777832031 -40.03169631958008 -32.11161804199219 -42.55473327636719 -42.55473327636719
31.845834732055664 -32.72394561767578 -40.11094665527344 -30.804439544677734 -41.685646057128906 -41.685646057128906
31.9233341217041 -32.75971984863281 -40.143089294433594 -33.26371383666992 -42.2865104675293 -42.28650665283203
31.96583366394043 -32.797977447509766 -40.160884857177734 -30.76951789855957 -45.088958740234375 -45.088958740234375
31.926668167114258 -32.44209289550781 -39.96980667114258 -33.583614349365234 -43.148094177246094 -43.148094177246094
31.96416664123535 -32.18138122558594 -39.702880859375 -32.60245132446289 -40.244239

...

29.91083335876465 -32.378116607666016 -39.68681335449219 -33.85338592529297 -44.936790466308594 -44.93678665161133
29.90333366394043 -32.26864242553711 -39.67426300048828 -31.508634567260742 -40.15697479248047 -40.1569709777832
29.977500915527344 -32.12543869018555 -39.534542083740234 -30.41761016845703 -43.22043991088867 -43.22043991088867
29.935001373291016 -31.935697555541992 -39.46505355834961 -32.79270935058594 -36.43585205078125 -36.435855865478516
29.859167098999023 -32.00830078125 -39.52329635620117 -30.882305145263672 -40.77830505371094 -40.7783088684082
29.944168090820312 -32.03199768066406 -39.4843864440918 -31.26634407043457 -36.60447311401367 -36.604469299316406
29.877500534057617 -32.10237121582031 -39.567195892333984 -32.878814697265625 -38.792747497558594 -38.792747497558594
30.030834197998047 -32.1513557434082 -39.58753204345703 -31.1339054107666 -39.24005126953125 -39.24005126953125
29.961668014526367 -32.003990173339844 -39.509971618652344 -30.78919219970703 -42.4432

30.03499984741211 -32.612789154052734 -39.94340133666992 -31.365610122680664 -39.2140007019043 -39.21400451660156
30.01583480834961 -32.58483123779297 -39.933895111083984 -31.896583557128906 -36.78672409057617 -36.78672409057617
30.095001220703125 -32.43879699707031 -39.8590087890625 -33.63424301147461 -36.31591796875 -36.315914154052734
29.961668014526367 -32.03809356689453 -39.669921875 -30.6563663482666 -35.45976257324219 -35.45975875854492
29.68833351135254 -31.821338653564453 -39.64264678955078 -32.29301452636719 -39.245765686035156 -39.24576187133789
29.95416831970215 -31.581615447998047 -39.49580001831055 -31.589000701904297 -41.38588333129883 -41.38588333129883
29.762500762939453 -31.64375114440918 -39.55814743041992 -32.51411819458008 -40.6534423828125 -40.653438568115234
29.68666648864746 -31.756120681762695 -39.678741455078125 -32.735328674316406 -41.376895904541016 -41.37689971923828
29.758333206176758 -32.06759262084961 -39.884334564208984 -30.07170295715332 -37.0531463623

29.676668167114258 -31.821979522705078 -39.637027740478516 -31.257461547851562 -39.226097106933594 -39.22609329223633
29.685834884643555 -31.821706771850586 -39.662353515625 -29.758352279663086 -40.61274719238281 -40.61275100708008
29.68000030517578 -31.859365463256836 -39.69550704956055 -30.96044158935547 -37.34503936767578 -37.34504318237305
30.030000686645508 -32.05259704589844 -39.83515930175781 -33.89833450317383 -33.621002197265625 -33.62100601196289
29.463333129882812 -32.25196838378906 -39.91733169555664 -33.474613189697266 -41.77998352050781 -41.77998733520508
29.575834274291992 -32.53646469116211 -40.061031341552734 -33.62370681762695 -42.37596893310547 -42.3759651184082
29.775001525878906 -32.613075256347656 -40.175865173339844 -32.862464904785156 -39.8309326171875 -39.8309326171875
29.781667709350586 -32.732688903808594 -40.18549728393555 -32.632015228271484 -37.7154541015625 -37.71546173095703
29.788333892822266 -32.646697998046875 -40.14878463745117 -34.98998260498047 -39

29.692501068115234 -31.789596557617188 -39.568519592285156 -31.998821258544922 -43.51365661621094 -43.51365661621094
29.788333892822266 -31.517986297607422 -39.34987258911133 -28.55966567993164 -41.855613708496094 -41.85560989379883
29.690834045410156 -31.59134864807129 -39.395408630371094 -32.172908782958984 -38.91230010986328 -38.91230010986328
29.885000228881836 -31.625593185424805 -39.39406204223633 -33.15495300292969 -42.71159362792969 -42.71159362792969
29.73833465576172 -31.791419982910156 -39.52421188354492 -33.45294952392578 -39.42462158203125 -39.424625396728516
29.746667861938477 -32.01200866699219 -39.64601135253906 -31.178741455078125 -38.10622787475586 -38.10622787475586
29.808334350585938 -32.098270416259766 -39.68836212158203 -32.824790954589844 -41.39906311035156 -41.39906311035156
29.90250015258789 -32.24399948120117 -39.78871536254883 -30.579055786132812 -37.99705505371094 -37.99705505371094
29.794166564941406 -32.32587432861328 -39.742549896240234 -31.96289825439453

29.633333206176758 -32.05224609375 -39.62017822265625 -31.504148483276367 -41.32354736328125 -41.323551177978516
29.456666946411133 -31.94721794128418 -39.61720657348633 -32.3916015625 -42.351234436035156 -42.35124206542969
29.643333435058594 -32.09489440917969 -39.69187545776367 -31.586952209472656 -39.495052337646484 -39.495052337646484
29.763334274291992 -32.22682189941406 -39.749855041503906 -32.886451721191406 -38.48134231567383 -38.48134231567383
29.662500381469727 -32.12222671508789 -39.777252197265625 -32.951812744140625 -34.704315185546875 -34.70431900024414
29.365833282470703 -32.12031173706055 -39.820587158203125 -33.38618469238281 -40.33112716674805 -40.33112335205078
29.80500030517578 -32.13824462890625 -39.80083084106445 -32.176849365234375 -40.939697265625 -40.939693450927734
29.489999771118164 -31.897409439086914 -39.69931411743164 -31.04751968383789 -42.60651397705078 -42.60650634765625
29.671667098999023 -31.832630157470703 -39.695743560791016 -31.27968978881836 -38.8